In [5]:
import pandas as pd

file_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\버스전용차로\분석데이터\기흥IC~신갈JC_차간간격_50-50_001\기흥IC~신갈JC_차간간격_50-50_001.fzp"  # 또는 .txt 등

df = pd.read_csv(file_path, sep=";", encoding="utf-8")

display(df)


,$VEHICLE:SIMSEC,NO,VEHTYPE,LANE\LINK\NO,LANE\INDEX,LANE,LNCHG,POS,SPEED,SPEEDDIFF,ACCELERATION,SAFDISTNET
0,901.0,431,100,13,2,13-2,None,717.109,48.86,0.00,0.27,14.76
1,901.0,433,100,13,1,13-1,None,726.429,46.62,0.00,-0.20,10.91
2,901.0,437,100,13,2,13-2,None,697.847,45.36,-3.82,0.36,13.87
3,901.0,438,100,13,2,13-2,None,678.242,45.21,1.34,-0.06,16.06
4,901.0,448,300,13,4,13-4,None,560.164,48.20,0.00,0.00,8.93
...,...,...,...,...,...,...,...,...,...,...,...,...
3368331,4500.0,6997,400,18,2,18-2,None,42.648,50.68,-2.62,-0.25,18.68
3368332,4500.0,6998,100,18,4,18-4,None,32.410,51.35,-1.27,-0.15,19.66
3368333,4500.0,6999,100,18,3,18-3,None,31.727,51.96,-1.34,0.16,11.26
3368334,4500.0,7000,100,18,1,18-1,None,31.359,52.29,1.51,-0.19,15.09
